In [1]:
import pandas as pd
from window_generator import generate_windows_pd as wg
from collections import Counter

cols = ['created','application_id','ref_type','ref_hash','click_hash','attributed','implicit','device_brand','device_model',
       'wifi', 'device_language']

installs = pd.read_csv('./data/installs.csv', usecols = cols)
installs.head()

/home/fedoremelianenko/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_brand,device_model,wifi,device_language
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,NaN,3.739127e+17,NaN,3.301378e+18
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,NaN,7.805539e+18,NaN,3.301378e+18
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,NaN,8.355496e+18,NaN,4.060930e+18
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,NaN,2.355772e+18,NaN,3.301378e+18
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,NaN,6.156971e+18,NaN,3.301378e+18


In [2]:
installs['created'] = pd.to_datetime(installs['created'])
installs['attributed'] = installs['attributed'].apply(lambda x: 1 if (x==True) else 0)
installs['implicit'] = installs['implicit'].apply(lambda x: 1 if (x==True) else 0)
installs['wifi'] = installs['wifi'].apply(lambda x: 1 if (x==True) else 0)

installs['number_of_installs'] = 1
installs.head()

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_brand,device_model,wifi,device_language,number_of_installs
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,0,1,NaN,3.739127e+17,0,3.301378e+18,1
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,0,0,NaN,7.805539e+18,0,3.301378e+18,1
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,0,1,NaN,8.355496e+18,0,4.060930e+18,1
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,0,1,NaN,2.355772e+18,0,3.301378e+18,1
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,0,0,NaN,6.156971e+18,0,3.301378e+18,1


In [3]:
window1, window2, window3, window4, window5, window6, window7 = wg(installs,'created')
window1.head()

/home/fedoremelianenko/FIUBA/Organizacion de datos/TPs/TP2/window_generator.py:18: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  window1 = window1.loc[df[str_timestamp] > INITIAL_DATE]


,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_brand,device_model,wifi,device_language,number_of_installs
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,0,1,NaN,8.355496e+18,0,4.060930e+18,1
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,0,1,NaN,2.355772e+18,0,3.301378e+18,1
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,0,0,NaN,6.156971e+18,0,3.301378e+18,1
5,2019-04-20 18:19:27.485,1,1494519392962156891,7522785771858684314,NaN,0,1,NaN,6.208879e+18,0,3.301378e+18,1
6,2019-04-20 03:40:21.239,1,1494519392962156891,7882044913917355073,NaN,0,1,NaN,4.566898e+18,0,4.060930e+18,1


In [4]:
def get_mode(values):
    c = Counter(values)
    return c.most_common(1)[0][0]

iw1 = window1.groupby('ref_hash').agg({'application_id':get_mode,'ref_type':get_mode,'attributed':'mean',
                                      'implicit':'mean','device_brand':get_mode, 'device_model':get_mode,
                                      'wifi':'mean','device_language':get_mode, 'number_of_installs':'sum'})
iw1.rename(columns={'application_id':'app_most_installed','ref_type':'ref_type_mode','implicit':'implicit_installs',
                   'device_brand':'most_used_brand','device_model':'most_used_model','wifi':'wifi_installs',
                   'device_language':'language_installs'}, inplace=True)
iw1.head()

,implicit_installs,most_used_model,ref_type_mode,wifi_installs,number_of_installs,attributed,language_installs,most_used_brand,app_most_installed
ref_hash,,,,,,,,,
41863526108385,0.5,1.658417e+18,1891515180541284343,0.0,4,0.0,6.977049e+18,6.115026e+18,65
135153013040192,0.0,4.981016e+18,1891515180541284343,0.0,1,0.0,3.301378e+18,NaN,155
186034136943920,1.0,NaN,1891515180541284343,0.0,1,0.0,NaN,NaN,210
365882020742330,0.0,1.805456e+18,1891515180541284343,0.0,1,0.0,6.977049e+18,3.083059e+17,302
519199987760489,0.0,1.295045e+18,1891515180541284343,1.0,1,0.0,6.977049e+18,8.461030e+18,187


In [5]:
iw2 = window2.groupby('ref_hash').agg({'application_id':get_mode,'ref_type':get_mode,'attributed':'mean',
                                      'implicit':'mean','device_brand':get_mode, 'device_model':get_mode,
                                      'wifi':'mean','device_language':get_mode, 'number_of_installs':'sum'})
iw2.rename(columns={'application_id':'app_most_installed','ref_type':'ref_type_mode','implicit':'implicit_installs',
                   'device_brand':'most_used_brand','device_model':'most_used_model','wifi':'wifi_installs',
                   'device_language':'language_installs'}, inplace=True)

iw3 = window3.groupby('ref_hash').agg({'application_id':get_mode,'ref_type':get_mode,'attributed':'mean',
                                      'implicit':'mean','device_brand':get_mode, 'device_model':get_mode,
                                      'wifi':'mean','device_language':get_mode, 'number_of_installs':'sum'})
iw3.rename(columns={'application_id':'app_most_installed','ref_type':'ref_type_mode','implicit':'implicit_installs',
                   'device_brand':'most_used_brand','device_model':'most_used_model','wifi':'wifi_installs',
                   'device_language':'language_installs'}, inplace=True)

iw4 = window4.groupby('ref_hash').agg({'application_id':get_mode,'ref_type':get_mode,'attributed':'mean',
                                      'implicit':'mean','device_brand':get_mode, 'device_model':get_mode,
                                      'wifi':'mean','device_language':get_mode, 'number_of_installs':'sum'})
iw4.rename(columns={'application_id':'app_most_installed','ref_type':'ref_type_mode','implicit':'implicit_installs',
                   'device_brand':'most_used_brand','device_model':'most_used_model','wifi':'wifi_installs',
                   'device_language':'language_installs'}, inplace=True)

iw5 = window5.groupby('ref_hash').agg({'application_id':get_mode,'ref_type':get_mode,'attributed':'mean',
                                      'implicit':'mean','device_brand':get_mode, 'device_model':get_mode,
                                      'wifi':'mean','device_language':get_mode, 'number_of_installs':'sum'})
iw5.rename(columns={'application_id':'app_most_installed','ref_type':'ref_type_mode','implicit':'implicit_installs',
                   'device_brand':'most_used_brand','device_model':'most_used_model','wifi':'wifi_installs',
                   'device_language':'language_installs'}, inplace=True)

iw6 = window6.groupby('ref_hash').agg({'application_id':get_mode,'ref_type':get_mode,'attributed':'mean',
                                      'implicit':'mean','device_brand':get_mode, 'device_model':get_mode,
                                      'wifi':'mean','device_language':get_mode, 'number_of_installs':'sum'})
iw6.rename(columns={'application_id':'app_most_installed','ref_type':'ref_type_mode','implicit':'implicit_installs',
                   'device_brand':'most_used_brand','device_model':'most_used_model','wifi':'wifi_installs',
                   'device_language':'language_installs'}, inplace=True)

iw7 = window7.groupby('ref_hash').agg({'application_id':get_mode,'ref_type':get_mode,'attributed':'mean',
                                      'implicit':'mean','device_brand':get_mode, 'device_model':get_mode,
                                      'wifi':'mean','device_language':get_mode, 'number_of_installs':'sum'})
iw7.rename(columns={'application_id':'app_most_installed','ref_type':'ref_type_mode','implicit':'implicit_installs',
                   'device_brand':'most_used_brand','device_model':'most_used_model','wifi':'wifi_installs',
                   'device_language':'language_installs'}, inplace=True)

In [6]:
iw1.to_csv('./data/features/install_features/if_w1.csv')
iw2.to_csv('./data/features/install_features/if_w2.csv')
iw3.to_csv('./data/features/install_features/if_w3.csv')
iw4.to_csv('./data/features/install_features/if_w4.csv')
iw5.to_csv('./data/features/install_features/if_w5.csv')
iw6.to_csv('./data/features/install_features/if_w6.csv')
iw7.to_csv('./data/features/install_features/if_w7.csv')